In [1]:
%matplotlib qt5

%load_ext autoreload
%autoreload 2

import mplcursors
import mplfinance as mpf
import pandas as pd
import os
from data_class import kucoin_data, kucoin_api
import json
from datetime import datetime, timedelta
import dateparser
from typing import Optional
from binance.exceptions import UnknownDateFormat
import pytz
import matplotlib.pyplot as plt
import logging
import copy 
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(filename=f'app.log', filemode='a', format='%(asctime)s >>>> %(process)d - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

current_directory = os.getcwd()
print(f"Current Directory: {current_directory}")

Current Directory: /mnt/pumacydrive/Projects/binancebot/2_Kucoin/crypto_finder


In [2]:
keys = json.load(open(f"{current_directory}/keys.json","rb"))
api_type = 'live'
kucoin_rest_api = kucoin_api(api_type, keys[api_type])

In [3]:
how_much_ago = '2 year ago'
default_configs = {            
            # "1hour":{
            #     "interval": "1hour",
            #     "startTime": how_much_ago,
            #     "endTime": "now",
            #     "df_kline": pd.DataFrame(),

            #     "get_rsi": True,
            #     "df_rsi": pd.DataFrame(),

            #     "get_score": False,
            #     "df_score": pd.DataFrame(),

            #     "get_volume": True,
            #     "df_volume": pd.DataFrame(),

            #     "get_stochastic": False,
            #     "df_stochs": pd.DataFrame(),

            #     "get_bollinger": False,
            #     "df_bollinger": pd.DataFrame(),

            #     "get_macd": True,
            #     "df_macd": pd.DataFrame()
            # },
            "1day":{
                "interval": "1day",
                "startTime": how_much_ago,
                "endTime": "now",
                "df_kline": pd.DataFrame(),

                "get_rsi": True,
                "df_rsi": pd.DataFrame(),

                "get_score": False,
                "df_score": pd.DataFrame(),

                "get_volume": True,
                "df_volume": pd.DataFrame(),

                "get_stochastic": False,
                "df_stochs": pd.DataFrame(),

                "get_bollinger": False,
                "df_bollinger": pd.DataFrame(),

                "get_macd": True,
                "df_macd": pd.DataFrame()
            },
            "1week":{
                "interval": "1week",
                "startTime": how_much_ago,
                "endTime": "now",
                "df_kline": pd.DataFrame(),

                "get_rsi": True,
                "df_rsi": pd.DataFrame(),

                "get_score": False,
                "df_score": pd.DataFrame(),

                "get_volume": True,
                "df_volume": pd.DataFrame(),

                "get_stochastic": False,
                "df_stochs": pd.DataFrame(),

                "get_bollinger": False,
                "df_bollinger": pd.DataFrame(),

                "get_macd": True,
                "df_macd": pd.DataFrame()
            }
            }

first_interval = default_configs[list(default_configs.keys())[0]]

## Fetch Data

In [8]:
symbol = "BEPRO-USDT"
local_thread = kucoin_data(
    app=None, 
    kucoin_api_obj=kucoin_rest_api, 
    symbol=symbol, 
    base_asset=symbol.split("-")[0], 
    quote_asset=symbol.split("-")[1],
    default_configs=copy.deepcopy(default_configs), 
    delay=1,
    timeframe_result='D'
    )

local_thread.max_duration_days = 730
local_thread.get_multiple_klines_data()
local_thread.generate_indicator()

if local_thread.combined_indicators.empty:
    raise Exception(f"Data unavailable for {symbol}")

df_indicator = local_thread.combined_indicators
df_indicator = df_indicator.sort_values("Open time", ascending=True)
cols_duplicates = [i for i in df_indicator.columns if i.split("_")[0] in ["Open", "Close", "High", "Low"]]

df_indicator = df_indicator.drop_duplicates(subset=cols_duplicates)
# df_indicator = df_indicator[len(df_indicator)//2:]

df_candlestick = df_indicator.copy()
df_candlestick['Open time'] = pd.to_datetime(df_candlestick['Open time'])

interval = '1day'
df_candlestick = df_candlestick.set_index('Open time')[[f'Open_{interval}', f'High_{interval}', f'Low_{interval}', f'Close_{interval}']]
df_candlestick.rename(columns={f'Open_{interval}':'Open', f'Close_{interval}':'Close', f'High_{interval}':'High', f'Low_{interval}':'Low'}, inplace=True)

print(df_indicator.shape)
display(df_indicator.head(5))
display(df_indicator.tail(5))

(17521, 67)


,Open time,Open_1hour,Close_1hour,candle_diff_1hour,Volume_1hour,rsi_1hour,stochastic_rsi_k_1hour,rsi_change_intensity_1hour,trend_rsi_1hour,slope_rsi_1hour,...,slope_rsi_change_intensity_1week,macd_1week,signal_line_1week,macd_histogram_pct_1week,trend_macd_1week,slope_macd_1week,trend_signal_line_1week,slope_signal_line_1week,trend_macd_histogram_pct_1week,slope_macd_histogram_pct_1week
17520,2022-11-26 00:00:00,0.001524,0.001524,0.0,13202.8523,61.764706,0.0,1.110223e-14,Insufficient data,0.0,...,0.0,0.0,0.0,400.0,Insufficient data,0.0,Insufficient data,0.0,Insufficient data,0.0
17519,2022-11-26 01:00:00,0.001524,0.001524,0.0,13202.8523,61.764706,0.0,1.110223e-14,Insufficient data,0.0,...,0.0,0.0,0.0,400.0,Insufficient data,0.0,Insufficient data,0.0,Insufficient data,0.0


,Open time,Open_1hour,Close_1hour,candle_diff_1hour,Volume_1hour,rsi_1hour,stochastic_rsi_k_1hour,rsi_change_intensity_1hour,trend_rsi_1hour,slope_rsi_1hour,...,slope_rsi_change_intensity_1week,macd_1week,signal_line_1week,macd_histogram_pct_1week,trend_macd_1week,slope_macd_1week,trend_signal_line_1week,slope_signal_line_1week,trend_macd_histogram_pct_1week,slope_macd_histogram_pct_1week
1,2024-11-24 23:00:00,0.001750,0.001748,-0.114416,92807.3968,72.820172,94.004177,2.709701e+00,Uptrend,-1.295788,...,-0.575618,-0.001551,-0.001679,7.584881,Uptrend,-0.00008,Uptrend,-0.000025,Uptrend,-3.250922
0,2024-11-25 00:00:00,0.001748,0.001748,0.000000,92.7859,72.820172,92.496784,2.220446e-14,Downtrend,1.014083,...,-0.575618,-0.001551,-0.001679,7.584881,Uptrend,-0.00008,Uptrend,-0.000025,Uptrend,-3.250922


In [9]:
# BEFORE BULL
data = df_indicator.copy()

display(data[['Open time', 'slope_stochastic_rsi_k_1week']].head(2))
display(data[['Open time', 'slope_stochastic_rsi_k_1week']].tail(2))

timeframe = '1week'
if timeframe in list(default_configs.keys()):
    data[f'is_macd_histogram_pct_uptrend_{timeframe}'] = data.apply(lambda x:x[f'slope_macd_histogram_pct_{timeframe}'] < 0, axis=1)
    data = data[data[f'is_macd_histogram_pct_uptrend_{timeframe}']==True]

    data[f'is_stochrsi_uptrend_{timeframe}'] = data.apply(lambda x:x[f'slope_stochastic_rsi_k_{timeframe}'] < -3, axis=1)
    data = data[data[f'is_stochrsi_uptrend_{timeframe}']==True]

    data[f'is_stochrsi_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] > 20, axis=1)
    data = data[data[f'is_stochrsi_high_{timeframe}']==True]

timeframe = '1day'
if timeframe in list(default_configs.keys()):
    data[f'is_stochrsi_uptrend_{timeframe}'] = data.apply(lambda x:x[f'slope_stochastic_rsi_k_{timeframe}'] < -3, axis=1)
    data = data[data[f'is_stochrsi_uptrend_{timeframe}']==True]

    data[f'is_stochrsi_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] > 20, axis=1)
    data = data[data[f'is_stochrsi_high_{timeframe}']==True]

    data[f'is_stochrsi_not_too_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] < 80, axis=1)
    data = data[data[f'is_stochrsi_not_too_high_{timeframe}']==True]

timeframe = '1hour'
if timeframe in list(default_configs.keys()):
    data[f'is_stochrsi_uptrend_{timeframe}'] = data.apply(lambda x:x[f'slope_stochastic_rsi_k_{timeframe}'] < -3, axis=1)
    data = data[data[f'is_stochrsi_uptrend_{timeframe}']==True]

    data[f'is_stochrsi_not_too_high_{timeframe}'] = data.apply(lambda x:x[f'stochastic_rsi_k_{timeframe}'] < 80, axis=1)
    data = data[data[f'is_stochrsi_not_too_high_{timeframe}']==True]

before_bull = data.copy()
print(before_bull.shape)

,Open time,slope_stochastic_rsi_k_1week
17520,2022-11-26 00:00:00,0.0
17519,2022-11-26 01:00:00,0.0


,Open time,slope_stochastic_rsi_k_1week
1,2024-11-24 23:00:00,-20.695896
0,2024-11-25 00:00:00,-20.695896


(50, 72)


In [10]:
if True:
    fig, ax = plt.subplots(2, 1, figsize=(16,9), sharex=True)
    fig.suptitle(f"Analysis_{symbol}", fontsize=16)
    axs = ax.flatten()

    interval = '1day'

    plt_ind = 0
    mpf.plot(
        df_candlestick,
        type='candle',
        ax=ax[plt_ind],  # Use the subplot axis
        style='yahoo',  # Style of the candlestick plot
        ylabel=f'{interval} Candlestick Plot',  # Add ylabel,
    )

    interval = '1week'

    plt_ind = 1
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_macd_histogram_pct_{interval}'], label=f'slope_macd_histogram_pct_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()
    
    for label in before_bull['Open time']:
        ax[0].axvline(x=label, color='green', linestyle='--', alpha=0.5)

else:
    fig, ax = plt.subplots(7, 1, figsize=(16,9), sharex=True)
    fig.suptitle(f"Analysis_{symbol}", fontsize=16)
    axs = ax.flatten()

    interval = '1day'

    plt_ind = 0
    mpf.plot(
        df_candlestick,
        type='candle',
        ax=ax[plt_ind],  # Use the subplot axis
        style='yahoo',  # Style of the candlestick plot
        ylabel=f'{interval} Candlestick Plot',  # Add ylabel,
    )

    interval = '1week'

    plt_ind = 1
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_macd_histogram_pct_{interval}'], label=f'slope_macd_histogram_pct_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    plt_ind = 2
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_stochastic_rsi_k_{interval}'], label=f'slope_stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    plt_ind = 3
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'stochastic_rsi_k_{interval}'], label=f'stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    interval = '1day'

    plt_ind = 4
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_macd_histogram_pct_{interval}'], label=f'slope_macd_histogram_pct_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    plt_ind = 5
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'slope_stochastic_rsi_k_{interval}'], label=f'slope_stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    plt_ind = 6
    ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'stochastic_rsi_k_{interval}'], label=f'stochastic_rsi_k_{interval}')
    ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    ax[plt_ind].legend()

    # plt_ind = 4
    # ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'macd_{interval}'], label=f'Open_{interval}', color='blue')
    # ax[plt_ind].plot(df_indicator['Open time'], df_indicator[f'signal_line_{interval}'], label=f'signal_line_{interval}', color='orange')
    # ax[plt_ind].axhline(linewidth=1, color='k', alpha=0.5)
    # ax[plt_ind].legend()

    mplcursors.cursor(ax[1].lines + ax[2].lines + ax[3].lines + ax[4].lines + ax[5].lines + ax[6].lines, hover=False)

    for label in before_bull['Open time']:
        ax[0].axvline(x=label, color='green', linestyle='--', alpha=0.5)

plt.xticks(rotation=45, fontsize=7) 
pass

: 